In [1]:
import pandas as pd 
import numpy as np
import pandasql as ps
import os
from preprocess_days_stats import preprocess_cumulative_stats, preprocess_match_days
import warnings
# Ignora tutti i warning temporaneamente
warnings.filterwarnings("ignore")


df_giornate = preprocess_match_days(r"c:\Users\Hp\Documents\Serie_A\csv_serie_a")

Reading file: I1 (0).csv
Reading file: I1 (1).csv
Reading file: I1 (10).csv
Reading file: I1 (11).csv
Reading file: I1 (12).csv
Reading file: I1 (13).csv
Reading file: I1 (14).csv
Reading file: I1 (15).csv
Reading file: I1 (16).csv
Reading file: I1 (17).csv
Reading file: I1 (18).csv
Reading file: I1 (2).csv
Reading file: I1 (3).csv
Reading file: I1 (4).csv
Reading file: I1 (5).csv
Reading file: I1 (6).csv
Reading file: I1 (7).csv
Reading file: I1 (8).csv
Reading file: I1 (9).csv
preprocessing finished!


In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,accuracy_score
from preprocess_features import preprocess_columns

for giorni_cumulativi in range(10):
    giorni_cumulativi = giorni_cumulativi+1
    df_Serie_A_days, stats_teams_serie_A = preprocess_cumulative_stats(dataframe=df_giornate, giorni_cumulativi=giorni_cumulativi)
    for numero_colonne in ['all','less','few']:    
        X_train_norm, X_valid_norm, X_test_norm, Train_labels_encoded, Valid_labels_encoded, Test_labels_encoded = preprocess_columns(
            df_Serie_A_days,numero_colonne,giorni_cumulativi)

        best_accuracy=0
        for vicini in range(3,40): #prendiamo diversi numeri di vicini possibili
            knn_model = KNeighborsClassifier(n_neighbors=3*vicini) #addestriamo il modello con questo numero di vicini 
            knn_model.fit(X_train_norm, Train_labels_encoded) 
            y_valid_test=knn_model.predict(X_valid_norm)
            accuracy=round(accuracy_score(Valid_labels_encoded, y_valid_test),2)

            if accuracy > best_accuracy: #cerco di capire quale numero di vicini restituisce il risultato migliore
                best_vicini = 5*vicini
                best_accuracy = accuracy

        print(f"il modello migliore per {giorni_cumulativi+1} giorni e {numero_colonne} features si ha con {best_vicini} vicini, con un'accuracy di {best_accuracy}")


le squadre sono uguali:
 [ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]
preprocess finished
utilizzando tutte le features
lunghezza dataframe train 5919, 
 lunghezza dataframe valid 345, 
 lunghezza dataframe test 696
il modello migliore per 0 giorni e all features si ha con 90 vicini, con un'accuracy di 0.52
utilizzando meno features
lunghezza dataframe train 5919, 
 lunghezza dataframe valid 345, 
 lunghezza dataframe test 696
il modello migliore per 0 giorni e less features si ha con 60 vicini, con un'accuracy di 0.54
utilizzando poche features
lunghezza dataframe train 5919, 
 lunghezza dataframe valid 345, 
 lunghezza dataframe test 696
il modello migliore per 0 giorni e few features si ha con 75 vicini, con un'accuracy di 0.54

le squadre sono uguali:
 [ True  True  

KeyboardInterrupt: 